In [ ]:
import sys
import os
# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Check if we need to change directory to the project root
if os.path.basename(current_dir) == 'src':
    project_root = os.path.dirname(current_dir)
    os.chdir(project_root)
    print(f"Changed working directory to: {project_root}")
    sys.path.append(current_dir)
else:
    scripts_path = os.path.join(current_dir, 'notebooks')
    if os.path.exists(scripts_path):
        sys.path.append(scripts_path)
# --- Step 1: Set project root dynamically ---
# Get the absolute path of the current notebook
notebook_path = os.path.abspath("")
current_dir = os.getcwd()

# Detect if we're in the notebooks folder or elsewhere
project_root = current_dir
if os.path.basename(current_dir) == "notebooks":
    project_root = os.path.dirname(current_dir)

# Add project root to sys.path so we can import src modules
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root set to: {project_root}")
print(f"Python sys.path: {sys.path[:3]} ...")  # Show first 3 entries for sanity
# --- Step 2: Import project modules ---
from src.data_preprocessing import DataPreprocessor
from src.eda import InsuranceEDA
# Load and preprocess the data
dp = DataPreprocessor(
    raw_path="D:/Python/Week-3/Raw_Data/MachineLearningRating_v3.txt",
    out_path="D:/Python/Week-3/Insurance-Analytics-Week-3-/data/processed/processed_MachineLearningRating_v3.csv"
)
df = dp.process(save_format="csv", create_features=True)

# Initialize EDA
eda = InsuranceEDA(df)


In [ ]:

# 1) Data Summarization
eda.data_structure_summary()
eda.descriptive_statistics()


In [ ]:

# 2) Data Quality
eda.missing_value_summary()


In [ ]:

# 3) Univariate Analysis
eda.plot_univariate_distributions()


In [ ]:

# 4) Bivariate Analysis
eda.correlation_analysis()
eda.monthly_change_scatter()


In [ ]:

# 5) Data Comparison / Trends
eda.compare_trends(feature="CoverType")


In [ ]:

# 6) Outlier Detection
eda.plot_outliers()


In [ ]:

# 7) Creative Visualizations
eda.creative_visualizations()
